In [63]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from langchain.agents import (AgentExecutor, create_tool_calling_agent, 
            create_openai_tools_agent, create_openai_functions_agent,create_xml_agent,
            create_json_chat_agent,create_structured_chat_agent, create_react_agent,
            create_self_ask_with_search_agent)
from langchain_community.tools.tavily_search import TavilySearchResults,TavilyAnswer
from langchain_core.prompts import ChatPromptTemplate
from langchain import hub
from langchain_core.tools import tool,StructuredTool

In [64]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [65]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llmOpenAI=ChatOpenAI(model="gpt-3.5-turbo")

<p>The core idea of agents is to use a language model to chose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents , a language model is used as a reasoning engine to determine which actions to take and in which order</p>

In [32]:
singlePrompt="Who is Lawrence Bishnoi?"
multiPrompt="In the Year 2024, what will be Sachin Tendulkar's age to the power 12?. Use both the tools: `tavily_search_results` as well as `exponentiate`"

<h3>Tool Calling with Agent</h3>

In [94]:
tools=[TavilySearchResults(name="tavily_search_results",max_results=2)]

In [114]:
def exponentiate(base:int,exponent:int) -> float:
    """
        Exponentiate one number with another and return the output
        base: The Base Number
        exponent: The times you would exponentiate with
    """
    return float(base)**float(exponent)

exponent=StructuredTool.from_function(
    func=exponentiate,
    name="exponentiate",
    description="Useful when you need to find the exponent of any given base number"
)

# For Multi Input Tools
multiInputTools=[TavilySearchResults(name="tavily_search_results",max_results=2),exponent]

In [115]:
exponent.name

'exponentiate'

In [116]:
exponent.invoke(input={'base':'52','exponent':'12'})

3.908770064862502e+20

In [205]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        ("system","You are a helpful assistant."),
        ("placeholder","{chat_history}"),
        ("human","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)

In [206]:
agent=create_tool_calling_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_tool_calling_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [207]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results` with `{'query': 'Lawrence Bishnoi'}`


[{'url': 'https://www.indiatoday.in/india/story/lawrence-bishnoi-journey-law-student-dreaded-gangster-salman-khan-threat-2625439-2024-10-30', 'content': "Lawrence Bishnoi: How a law student became a dreaded gangster - India Today Lawrence Bishnoi: How a law student became a dreaded gangster India Today's Arvind Ojha gives us a detailed insight into Bishnoi's journey. Lawrence Bishnoi, the dreaded gangster who rose to prominence for making numerous threat calls to Bollywood actor Salman Khan and also for plotting the murder of Punjabi singer Sidhu Moose Wala, has returned to the limelight again. India Today's Arvind Ojha gives us a detailed insight into Bishnoi's journey during a podcast with Aaj Tak Radio. India Today TV's Arvind Ojha revealed during the podcast that when he spoke to Goldy Brar, the latter confirmed that Salman Khan was the Bishnoi gang's target."}, {'url':

{'input': 'Who is Lawrence Bishnoi?',
 'output': 'Lawrence Bishnoi is a dreaded gangster who was once a law student. He gained notoriety for making numerous threat calls to Bollywood actor Salman Khan and for plotting the murder of Punjabi singer Sidhu Moose Wala. Lawrence Bishnoi, a 31-year-old law graduate from Punjab, has been accused by the NIA of running a trans-national crime syndicate. He has been in jail since 2015.'}

In [208]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results` with `{'query': 'Sachin Tendulkar age in 2024'}`


[{'url': 'https://sports.ndtv.com/cricket/sachin-tendulkar-celebrates-50th-birthday-the-a-to-z-of-master-blaster-3972224', 'content': "Sachin Tendulkar Celebrates 50th Birthday: The A To Z Of 'Master Blaster' | Cricket News Sports Home Cricket News Sachin Tendulkar Celebrates 50th Birthday The A To Z Of Master Blaster H (Harris Shield) The famous Mumbai Inter-school cricket tournament, where the world first came to know about Sachin Ramesh Tendulkar after his 664-run stand with Vinod Kambli. N (Narsingh Deonarine) Narsingh Deonarine's international career is nothing to write home about but the West Indian cricketer will forever cherish being the last-ever bowler to dismiss Tendulkar in international cricket -- during the second Test at the Wankhede Stadium in 2013. India Cricket Team Mumbai Indians Sachin Tendulkar Cricket Get the Latest Cricket Updates, Check B

{'input': "In the Year 2024, what will be Sachin Tendulkar's age to the power 12?. Use both the tools: `tavily_search_results` as well as `exponentiate`",
 'output': "In the year 2024, Sachin Tendulkar will be 50 years old. Now, if we calculate Sachin Tendulkar's age to the power of 12, it will be approximately 1.16 x 10^20."}

<h3> OpenAI Tools with Agents </h3>

In [209]:
prompt=hub.pull("hwchase17/openai-tools-agent")

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [210]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [211]:
agent=create_openai_tools_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_openai_tools_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [212]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results` with `{'query': 'Lawrence Bishnoi'}`


[{'url': 'https://www.reuters.com/world/who-is-lawrence-bishnoi-centre-row-between-india-canada-2024-10-15/', 'content': 'WHO IS LAWRENCE BISHNOI? The NIA has accused the 31-year-old law graduate, in jail since 2015, of running a trans-national crime syndicate. Born in the northern state of Punjab, Bishnoi is short'}, {'url': 'https://www.bbc.com/news/articles/c4gqyxwl27no', 'content': 'The Canadian police were referring to Lawrence Bishnoi, a 31-year-old gangster from India, now back in the spotlight domestically and internationally. Indian police say his gang is allegedly linked to the killing of a prominent politician in Mumbai at the weekend - gunmen shot dead 66-year-old Baba Siddique near his son’s office. Once among India’s most wanted, Bishnoi has been in prison since 2015, now held far from his native Punjab state in Gujarat. Indian police link Bishnoi\'s gang to B

{'input': 'Who is Lawrence Bishnoi?',
 'output': "Lawrence Bishnoi is a 31-year-old gangster from India. He is accused of running a trans-national crime syndicate and has been in jail since 2015. Bishnoi, originally from Punjab, has been in the spotlight domestically and internationally for his alleged involvement in criminal activities. Indian police link Bishnoi's gang to various criminal activities, including the killing of a prominent politician in Mumbai. He has attracted followers due to his money, style, and influence. If you need more information, feel free to ask."}

In [213]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 52, 'exponent': 12}`


3.908770064862502e+20
Invoking: `tavily_search_results` with `{'query': 'Sachin Tendulkar age in 2024'}`


[{'url': 'https://celebrityborns.com/biography/sachin-tendulkar/2586', 'content': '14. minutes. 05. seconds. 51 years, 5months, 26 days old age Sachin Tendulkar will turn 52 on 24 April, 2025. Only 6 months, 3 days, 7 hours,15 minutes has left for his next birthday. Sachin Tendulkar Born On.'}, {'url': 'https://wikibio.in/sachin-tendulkar/', 'content': 'As I stood there watching the Indian greats on the field, I kept telling myself that I needed to be a part of the action in the middle.”\n14-year-old Sachin Tendulkar as a ball boy during the 1987 Cricket World Cup\nSachin Tendulkar with Vinod Kambli in his school days\nSachin Tendulkar praying to Lord Ganesha\nSachin Tendulkar’s restaurant\nSachin Tendulkar with Bharat Ratna\nSachin Tendulkar with Michael Schumacher\nSachin Tend

{'input': "In the Year 2024, what will be Sachin Tendulkar's age to the power 12?. Use both the tools: `tavily_search_results` as well as `exponentiate`",
 'output': "Sachin Tendulkar will turn 52 on April 24, 2025. \n\nIn the year 2024, Sachin Tendulkar's age to the power of 12 will be approximately 3.91 x 10^20."}

<h3> OpenAI Functions with Agents (Use OpenAI Tools Agent in lieu of this) </h3>

In [214]:
prompt=hub.pull("hwchase17/openai-functions-agent")

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [215]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [216]:
agent=create_openai_functions_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_openai_functions_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [217]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results` with `{'query': 'Lawrence Bishnoi'}`


[{'url': 'https://www.indiatoday.in/india/story/lawrence-bishnoi-journey-law-student-dreaded-gangster-salman-khan-threat-2625439-2024-10-30', 'content': "Lawrence Bishnoi: How a law student became a dreaded gangster - India Today Lawrence Bishnoi: How a law student became a dreaded gangster India Today's Arvind Ojha gives us a detailed insight into Bishnoi's journey. Lawrence Bishnoi, the dreaded gangster who rose to prominence for making numerous threat calls to Bollywood actor Salman Khan and also for plotting the murder of Punjabi singer Sidhu Moose Wala, has returned to the limelight again. India Today's Arvind Ojha gives us a detailed insight into Bishnoi's journey during a podcast with Aaj Tak Radio. India Today TV's Arvind Ojha revealed during the podcast that when he spoke to Goldy Brar, the latter confirmed that Salman Khan was the Bishnoi gang's target."}, {'url':

{'input': 'Who is Lawrence Bishnoi?',
 'output': 'Lawrence Bishnoi is a dreaded gangster who gained notoriety for making numerous threat calls to Bollywood actor Salman Khan and plotting the murder of Punjabi singer Sidhu Moose Wala. Lawrence Bishnoi, who was a law student, has a criminal background and is currently serving time in Sabarmati Central Jail at Ahmedabad. Recently, he was elected as the national president of the youth wing of the All India Animal Protection Bishnoi Samaj. If you would like to learn more, you can read the articles [here](https://www.indiatoday.in/india/story/lawrence-bishnoi-journey-law-student-dreaded-gangster-salman-khan-threat-2625439-2024-10-30) and [here](https://www.newindianexpress.com/nation/2024/Oct/30/gangster-lawrence-bishnoi-elected-as-youth-wing-head-of-all-india-animalprotection-bishnoi-samaj).'}

In [218]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results` with `{'query': 'Sachin Tendulkar age in 2024'}`


[{'url': 'https://wikibio.in/sachin-tendulkar/', 'content': 'As I stood there watching the Indian greats on the field, I kept telling myself that I needed to be a part of the action in the middle.”\n14-year-old Sachin Tendulkar as a ball boy during the 1987 Cricket World Cup\nSachin Tendulkar with Vinod Kambli in his school days\nSachin Tendulkar praying to Lord Ganesha\nSachin Tendulkar’s restaurant\nSachin Tendulkar with Bharat Ratna\nSachin Tendulkar with Michael Schumacher\nSachin Tendulkar writing with his left hand\nSachin Tendulkar with his wax statue at Madame Tussauds\nSachin Tendulkar with his Cricket Kit\nSachin Tendulkar’s Instagram post\nSachin Tendulkar Spartan Sports Controversy\nLove and passion for the game always helps you find new ways to practice, and above all to enjoy what you do.#FlashbackFriday pic.twitter.com/7UHH13fe0Q\n— Sachin Tendulk

{'input': "In the Year 2024, what will be Sachin Tendulkar's age to the power 12?. Use both the tools: `tavily_search_results` as well as `exponentiate`",
 'output': "In the year 2024, Sachin Tendulkar will be 51 years old. If we calculate Sachin Tendulkar's age to the power of 12, the result is approximately 309,629,344,375,621,400."}

<h3>XML Agent</h3>

In [150]:
prompt=hub.pull(owner_repo_commit="hwchase17/xml-agent-convo")

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [151]:
print(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tools'], input_types={}, partial_variables={}, template="You are a helpful assistant. Help the user answer any questions.\n\nYou have access to the following tools:\n\n{tools}\n\nIn order to use a tool, you can use <tool></tool> and <tool_input></tool_input> tags. You will then get back a response in the form <observation></observation>\nFor example, if you have a tool called 'search' that could run a google search, in order to search for the weather in SF you would respond:\n\n<tool>search</tool><tool_input>weather in SF</tool_input>\n<observation>64 degrees</observation>\n\nWhen you are done, respond with a final answer between <final_answer></final_answer>. For example:\n\n<final_answer>The weather in SF is 64 degrees</final_answer>\n\nBegin!\n\nPrevious Conversation:\n{chat_history}\n\nQuestion: {input}\n{agent_scratchpad}"), additional_kwargs={})]


In [152]:
agent=create_xml_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_xml_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [153]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...
<tool>tavily_search_results</tool><tool_input>Lawrence Bishnoi[{'url': 'https://www.indiatoday.in/india/story/lawrence-bishnoi-journey-law-student-dreaded-gangster-salman-khan-threat-2625439-2024-10-30', 'content': "Lawrence Bishnoi: How a law student became a dreaded gangster - India Today Lawrence Bishnoi: How a law student became a dreaded gangster India Today's Arvind Ojha gives us a detailed insight into Bishnoi's journey. Lawrence Bishnoi, the dreaded gangster who rose to prominence for making numerous threat calls to Bollywood actor Salman Khan and also for plotting the murder of Punjabi singer Sidhu Moose Wala, has returned to the limelight again. India Today's Arvind Ojha gives us a detailed insight into Bishnoi's journey during a podcast with Aaj Tak Radio. India Today TV's Arvind Ojha revealed during the podcast that when he spoke to Goldy Brar, the latter confirmed that Salman Khan was the Bishnoi gang's target."}, {'url': 'https://ww

{'input': 'Who is Lawrence Bishnoi?',
 'output': 'Lawrence Bishnoi is a dreaded gangster who rose to prominence for making numerous threat calls to Bollywood actor Salman Khan and plotting the murder of Punjabi singer Sidhu Moose Wala. He has also been elected as the national president of the youth wing of the All India Animal Protection Bishnoi Samaj.'}

In [154]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...
<tool>exponentiate</tool><tool_input>2024, 12

ValidationError: 2 validation errors for exponentiate
base
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='2024, 12', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing
exponent
  Field required [type=missing, input_value={'base': '2024, 12'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing

<h3>JSON Chat Agent</h3>

In [173]:
prompt=hub.pull(owner_repo_commit="hwchase17/react-chat-json")     

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [174]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations a

In [175]:
agent=create_json_chat_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_json_chat_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [176]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...
```json
{
    "action": "tavily_search_results",
    "action_input": "Lawrence Bishnoi"
}
```[{'url': 'https://www.indiatoday.in/india/story/lawrence-bishnoi-journey-law-student-dreaded-gangster-salman-khan-threat-2625439-2024-10-30', 'content': "Lawrence Bishnoi: How a law student became a dreaded gangster - India Today Lawrence Bishnoi: How a law student became a dreaded gangster India Today's Arvind Ojha gives us a detailed insight into Bishnoi's journey. Lawrence Bishnoi, the dreaded gangster who rose to prominence for making numerous threat calls to Bollywood actor Salman Khan and also for plotting the murder of Punjabi singer Sidhu Moose Wala, has returned to the limelight again. India Today's Arvind Ojha gives us a detailed insight into Bishnoi's journey during a podcast with Aaj Tak Radio. India Today TV's Arvind Ojha revealed during the podcast that when he spoke to Goldy Brar, the latter confirmed that Salman Khan was the Bishnoi gang's

{'input': 'Who is Lawrence Bishnoi?',
 'output': 'Lawrence Bishnoi is a dreaded gangster who has gained notoriety for making threats to Bollywood actor Salman Khan and being involved in various criminal activities. He was elected as the national president of the youth wing of the All India Animal Protection Bishnoi Samaj while being in jail.'}

In [177]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...
```json
{
    "action": "Final Answer",
    "action_input": "Sachin Tendulkar's age to the power 12 in the year 2024 will be 161051.648"
}
```

> Finished chain.


{'input': "In the Year 2024, what will be Sachin Tendulkar's age to the power 12?. Use both the tools: `tavily_search_results` as well as `exponentiate`",
 'output': "Sachin Tendulkar's age to the power 12 in the year 2024 will be 161051.648"}

<h3>Structured Chat (Capable of Taking MultiInput Tools)</h3>

In [187]:
prompt=hub.pull(owner_repo_commit="hwchase17/structured-chat-agent")     

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [188]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'], input_types={}, partial_variables={}, template='Respond to the human as helpfully and accurately as possible. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per $JSON_BLOB, as shown:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nFollow this format:\n\nQuestion: input question to answer\nThought: consider previous and subsequent steps\nAction:\n```\n$JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. R

In [197]:
agent=create_structured_chat_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_structured_chat_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [198]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: Action:
```
{
  "action": tavily_search_results,
  "action_input": "Lawrence Bishnoi"
}
```
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE

In [199]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "tavily_search_results",
  "action_input": "Sachin Tendulkar age in 2024"
}
```[{'url': 'https://wikibio.in/sachin-tendulkar/', 'content': 'As I stood there watching the Indian greats on the field, I kept telling myself that I needed to be a part of the action in the middle.”\n14-year-old Sachin Tendulkar as a ball boy during the 1987 Cricket World Cup\nSachin Tendulkar with Vinod Kambli in his school days\nSachin Tendulkar praying to Lord Ganesha\nSachin Tendulkar’s restaurant\nSachin Tendulkar with Bharat Ratna\nSachin Tendulkar with Michael Schumacher\nSachin Tendulkar writing with his left hand\nSachin Tendulkar with his wax statue at Madame Tussauds\nSachin Tendulkar with his Cricket Kit\nSachin Tendulkar’s Instagram post\nSachin Tendulkar Spartan Sports Controversy\nLove and passion for the game always helps you find new ways to practice, and above all to enjoy what you do.#FlashbackFriday pic.twitter.com/7UHH13fe0

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: Action:
```
{
  "action": "exponentiate",
  "action_input": {
    "base": 2024 - 1973,
    "exponent": 12
  }
}
```
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE

<h3>React Agent</h3>

In [200]:
prompt=hub.pull(owner_repo_commit="hwchase17/react")

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\langsmith\client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [201]:
prompt.get_prompts()

[PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')]

In [202]:
agent=create_react_agent(llm=llmOpenAI,tools=tools,prompt=prompt)
agentExecutor=AgentExecutor(agent=agent,tools=tools,verbose=True)

multiInputAgent=create_react_agent(llm=llmOpenAI,tools=multiInputTools,prompt=prompt)
multiAgentExecutor=AgentExecutor(agent=multiInputAgent,tools=multiInputTools,verbose=True)

In [203]:
responseSingleTool=agentExecutor.invoke(input={"input":singlePrompt})
responseSingleTool



> Entering new AgentExecutor chain...
I should use tavily_search_results to find information about Lawrence Bishnoi.
Action: tavily_search_results
Action Input: Lawrence Bishnoi[{'url': 'https://www.indiatoday.in/india/story/lawrence-bishnoi-journey-law-student-dreaded-gangster-salman-khan-threat-2625439-2024-10-30', 'content': "Lawrence Bishnoi: How a law student became a dreaded gangster - India Today Lawrence Bishnoi: How a law student became a dreaded gangster India Today's Arvind Ojha gives us a detailed insight into Bishnoi's journey. Lawrence Bishnoi, the dreaded gangster who rose to prominence for making numerous threat calls to Bollywood actor Salman Khan and also for plotting the murder of Punjabi singer Sidhu Moose Wala, has returned to the limelight again. India Today's Arvind Ojha gives us a detailed insight into Bishnoi's journey during a podcast with Aaj Tak Radio. India Today TV's Arvind Ojha revealed during the podcast that when he spoke to Goldy Brar, the latter con

{'input': 'Who is Lawrence Bishnoi?',
 'output': 'Lawrence Bishnoi is a notorious gangster who has been involved in criminal activities and has a position in the All India Animal Protection Bishnoi Samaj.'}

In [204]:
responseMultiTool=multiAgentExecutor.invoke(input={"input":multiPrompt})
responseMultiTool



> Entering new AgentExecutor chain...
I need to find out Sachin Tendulkar's age in 2024 and then calculate it to the power of 12.
Action: tavily_search_results
Action Input: Sachin Tendulkar age in 2024[{'url': 'https://www.cricbuzz.com/profiles/25/sachin-tendulkar', 'content': 'CSA Four-Day Series Division One 2024-25. Western Province vs Dolphins ... it was the popular opinion that Tendulkar was ready, at the tender age of 16, to be playing international cricket. He was'}, {'url': 'https://sports.ndtv.com/cricket/sachin-tendulkar-celebrates-50th-birthday-the-a-to-z-of-master-blaster-3972224', 'content': "Sachin Tendulkar Celebrates 50th Birthday: The A To Z Of 'Master Blaster' | Cricket News Sports Home Cricket News Sachin Tendulkar Celebrates 50th Birthday The A To Z Of Master Blaster H (Harris Shield) The famous Mumbai Inter-school cricket tournament, where the world first came to know about Sachin Ramesh Tendulkar after his 664-run stand with Vinod Kambli. N (Narsingh Deonarine)

ValidationError: 2 validation errors for exponentiate
base
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='51, 12', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing
exponent
  Field required [type=missing, input_value={'base': '51, 12'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing